[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [34]:
# Import pandas, numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
# Read dataset with extra params sep='\t', encoding="latin-1"
dt = pd.read_csv('labeledTrainData.tsv', sep='\t', encoding="latin-1")

In [36]:
dt.sample(8)

,id,sentiment,review
23974,3148_9,1,I love this show. Period. I haven't been watch...
15824,5536_1,0,Revolt of the Zombies has no redeeming feature...
2342,2068_8,1,1958. The sleepy small Southern town of Clarks...
14892,265_7,1,"A series of random, seemingly insignificant th..."
20311,1963_4,0,"this movie is similar to Darkness Falls,and Th..."
1451,5843_3,0,Altered Species starts one Friday night in Los...
17833,144_8,1,Wow... 5 more hours of Riget. Lars continues t...
16917,280_2,0,"God, I was bored out of my head as I watched t..."


In [37]:
dt.describe(include="all")

,id,sentiment,review
count,25000,25000.00000,25000
unique,25000,NaN,24904
top,2582_8,NaN,How has this piece of crap stayed on TV this l...
freq,1,NaN,3
mean,NaN,0.50000,NaN
std,NaN,0.50001,NaN
min,NaN,0.00000,NaN
25%,NaN,0.00000,NaN
50%,NaN,0.50000,NaN
75%,NaN,1.00000,NaN


In [38]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [39]:
dt.shape

(25000, 3)

In [40]:
len(dt[dt['sentiment'] == 1])

12500

In [41]:
len(dt[dt['sentiment'] == 0])

12500

data has equal distribution

## 2. Preprocessing

In [42]:
# stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
# Removing special characters and "trash"
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text) # Your code here
    cleanr = re.compile('<.*?>')
    cleantxt = re.sub(cleanr, '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [44]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

porter = PorterStemmer()

# write a function called `tokenizer()` that split a text into list of words
def tokenizer(text):
    return word_tokenize(text)


# write a function named `tokenizer_porter()` that split a text into list of words and apply stemming technic
# Hint: porter.stem(word)
def tokenizer_porter(text):
    token = [porter.stem(word) for word in word_tokenize(text)]  
    return token

# Testing
print(tokenizer('Hi there, I am loving this, like with a lot of love'))
print(tokenizer_porter('Hi there, I am loving this, like with a lot of love'))

['Hi', 'there', ',', 'I', 'am', 'loving', 'this', ',', 'like', 'with', 'a', 'lot', 'of', 'love']
['Hi', 'there', ',', 'I', 'am', 'love', 'thi', ',', 'like', 'with', 'a', 'lot', 'of', 'love']


In [45]:
# split the dataset in train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dt['review'], dt['sentiment'], test_size = 0.2, random_state = 1)

In [46]:
len(X_train), len(X_test)

(20000, 5000)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [47]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(stop_words = stop, tokenizer = tokenizer_porter, preprocessor = preprocessor)

clf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

## 4. Evaluate Model

In [48]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def pred(predicted,compare):
    cm = pd.crosstab(compare,predicted)
    TN = cm.iloc[0,0]
    FN = cm.iloc[1,0]
    TP = cm.iloc[1,1]
    FP = cm.iloc[0,1]
    print("Confusion matrix:")
    print(cm)
    print()
    
    ##check accuracy of model
    print('Classification paradox:')
    print('Accuracy: ', round(((TP+TN)*100)/(TP+TN+FP+FN),2))
    print('False Negative Rate: ',round((FN*100)/(FN+TP),2))
    print('False Postive Rate: ',round((FP*100)/(FP+TN),2))
    print()
    print(classification_report(compare,predicted))



In [49]:
prediction = clf.predict(X_test)
pred(prediction, y_test)

Confusion matrix:
col_0         0     1
sentiment            
0          2200   303
1           237  2260

Classification paradox:
Accuracy:  89.2
False Negative Rate:  9.49
False Postive Rate:  12.11

             precision    recall  f1-score   support

          0       0.90      0.88      0.89      2503
          1       0.88      0.91      0.89      2497

avg / total       0.89      0.89      0.89      5000



## 5. Export Model 

In [50]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open('logisticRegression.pkl', 'wb'), protocol=4)